# 570. Managers with at Least 5 Direct Reports

SQL SchemaPandas SchemaTable: Employee+-------------+---------+| Column Name | Type    |+-------------+---------+| id          | int     || name        | varchar || department  | varchar || managerId   | int     |+-------------+---------+id is the primary key (column with unique values) for this table.Each row of this table indicates the name of an employee, their department, and the id of their manager.If managerId is null, then the employee does not have a manager.No employee will be the manager of themself. Write a solution to find managers with at least five direct reports.Return the result table in any order.The result format is in the following example. **Example 1:**Input: Employee table:+-----+-------+------------+-----------+| id  | name  | department | managerId |+-----+-------+------------+-----------+| 101 | John  | A          | null      || 102 | Dan   | A          | 101       || 103 | James | A          | 101       || 104 | Amy   | A          | 101       || 105 | Anne  | A          | 101       || 106 | Ron   | B          | 101       |+-----+-------+------------+-----------+Output: +------+| name |+------+| John |+------+

## Solution Explanation
This problem asks us to find managers who have at least five direct reports. In the Employee table, the `managerId` column indicates the manager of each employee. We need to:1. Group employees by their manager IDs2. Count how many employees report to each manager3. Filter for managers who have at least 5 direct reports4. Join back to the Employee table to get the manager's nameThe approach is to use SQL aggregation with GROUP BY to count employees per manager, then filter with HAVING, and finally join back to get the manager's name.

In [ ]:
import pandas as pddef find_managers(employee: pd.DataFrame) -> pd.DataFrame:    # Group by managerId and count the number of direct reports    manager_counts = employee.groupby('managerId').size().reset_index(name='count')        # Filter for managers with at least 5 direct reports    managers_with_five_reports = manager_counts[manager_counts['count'] >= 5]        # Join with the Employee table to get the manager's name    result = pd.merge(        managers_with_five_reports,        employee,        left_on='managerId',        right_on='id'    )[['name']]        return result

## Time and Space Complexity
* *Time Complexity**: O(n), where n is the number of employees in the table. We need to iterate through all employees once to group them by manager, and then perform a merge operation which is also O(n) in the worst case.* *Space Complexity**: O(n) in the worst case. We create intermediate DataFrames:* `manager_counts`: In the worst case, every employee could be a manager, so this could be O(n)* `managers_with_five_reports`: This is at most the size of `manager_counts`, so O(n)* `result`: This contains only managers with at least 5 reports, so it's bounded by O(n)Overall, the space complexity is O(n).

## Test Cases


In [ ]:
import pandas as pdimport unittestclass TestFindManagers(unittest.TestCase):    def test_example_case(self):        # Example case from the problem        data = {            'id': [101, 102, 103, 104, 105, 106],            'name': ['John', 'Dan', 'James', 'Amy', 'Anne', 'Ron'],            'department': ['A', 'A', 'A', 'A', 'A', 'B'],            'managerId': [None, 101, 101, 101, 101, 101]        }        employee = pd.DataFrame(data)        result = find_managers(employee)                # John should be the only manager with 5 direct reports        self.assertEqual(len(result), 1)        self.assertEqual(result.iloc[0]['name'], 'John')        def test_no_managers_with_five_reports(self):        # Test case where no manager has 5 direct reports        data = {            'id': [101, 102, 103, 104, 105, 106],            'name': ['John', 'Dan', 'James', 'Amy', 'Anne', 'Ron'],            'department': ['A', 'A', 'A', 'A', 'A', 'B'],            'managerId': [None, 101, 101, 101, 101, None]  # John has only 4 reports        }        employee = pd.DataFrame(data)        result = find_managers(employee)                # No manager should have 5 direct reports        self.assertEqual(len(result), 0)        def test_multiple_managers_with_five_reports(self):        # Test case with multiple managers having 5+ direct reports        data = {            'id': list(range(1, 16)),            'name': ['M1', 'E1', 'E2', 'E3', 'E4', 'E5', 'M2', 'E6', 'E7', 'E8', 'E9', 'E10', 'E11', 'E12', 'E13'],            'department': ['A'] * 15,            'managerId': [None, 1, 1, 1, 1, 1, None, 7, 7, 7, 7, 7, 7, 7, 7]  # M1 has 5 reports, M2 has 8        }        employee = pd.DataFrame(data)        result = find_managers(employee)                # Both M1 and M2 should be in the result        self.assertEqual(len(result), 2)        manager_names = set(result['name'].tolist())        self.assertEqual(manager_names, {'M1', 'M2'})if __name__ == '__main__':    unittest.main()